In [117]:
import os
import sys
import fnmatch
import pandas as pd
import operator
import shutil
import re
import javalang
def create_dev_test_train_split_and_vocabulary(root_path, 
                                               train_output, 
                                               vocabFile
                                              ):

    train_file = ''
    dev_file = ''
    test_file = ''

    word_counts = dict()
    
    for root, dirnames, filenames in os.walk(root_path):
        for filename in fnmatch.filter(filenames, '*.csv'):

            path = os.path.join(root, filename)

            if filename.endswith("test.csv"):
                test_file = path

            elif filename.endswith("dev.csv"):
                dev_file = path

            else:
                path = "csv/oneLineCode.csv"
                train_file = path
                dataframe = pd.read_csv(path, na_filter = False)
                for i,data in dataframe.iterrows():
                    d = splitComment(data["comment"])
                    c = tokenizeJavaCode(data["code"])
                    print(i)
                    print("comment:")
                    print(d)
                    
                    print("code:")
                    print(c)
                    s = []
                    s =  d+c
                    add_counts(word_counts, s)
                with open(path, 'r', encoding='utf-8') as text:
                    for line in text:
                        add_counts(word_counts, line)

    vocabulary = build_vocabulary(word_counts)
    write_vocabulary(vocabulary, vocabFile)

    write_processed_dataset(train_file, train_output,vocabFile)
#     write_processed_dataset(dev_txt_files, dev_output)
#     write_processed_dataset(test_txt_files, test_output)

def write_processed_dataset(input_file, output_file, vocabFile):
    names = [ 'comment', 'code','non-information']
    df = pd.DataFrame()
    word_vocabulary = read_vocabulary(vocabFile)
    dataframe = pd.read_csv(input_file, na_filter = False)
    for i,d in dataframe.iterrows():
        comment = []
        code = []
        label = 1 if d["non-information"] == "yes" else 0
        for token in splitComment(d["comment"]):
            comment.append(word_vocabulary.get(token,0))
        for token in tokenizeJavaCode(d["code"]):
            code.append(word_vocabulary.get(token,0))
        da = [{
            "comment":comment,
            "code":code,
            "label":label,
        }]
        print(da)
        df = df.append(da,ignore_index=True,sort=False)
    df.to_csv(output_file, index=False)

In [118]:
delimiters = "#", ".",",","<b>","</b>","-",":","<br>","_","?"," ",";"
def splitComment(string,delimiters = delimiters, maxsplit=0):
## replace all web https to https
    for l in string.split():
        if l.startswith("https"):
            string = string.replace(l,"https")
    ## split string by delimiters
    regexPattern = '|'.join(map(re.escape, delimiters))
    result =  re.split(regexPattern, string, maxsplit)
    ## split string by uppercase
    f = []
    for r in result:
        p =  re.sub(r'((?<=[a-z])[A-Z]|(?<!\A)[A-Z](?=[a-z]))', r' \1', r)
#         print(p)
        for q in p.split():
            if q != "":
                f.append(q)
    return f
def tokenizeJavaCode(code):
    result = []
    try:
        tokens = list(javalang.tokenizer.tokenize(code))
        for token in tokens:
            result.append(token.value)
        return result
    except:
        return list(code)
#TODO stopwords
STOPWORDS=["and","or","i"]
def iterable_to_dict(arr):
    return dict((x.strip(), i) for (i, x) in enumerate(arr))

def read_vocabulary(file_name):
    with open(file_name, 'r', encoding='utf-8') as f:
        return iterable_to_dict(f.readlines())
def add_counts(word_counts, line):
    for w in line:
        if w in STOPWORDS:
            continue
        word_counts[w] = word_counts.get(w, 0) + 1
END = "</S>"
UNK = "<UNK>"
NUM = "<NUM>"

def dump(d, path):
    with open(path, 'w') as f:
        for s in d:
            f.write("%s\n" % repr(s))
        
def write_vocabulary(vocabulary, file_name):
    if END not in vocabulary:
        vocabulary.append(END)
    if UNK not in vocabulary:
        vocabulary.append(UNK)

    print("Vocabulary size: %d" % len(vocabulary))

    with open(file_name, 'w', encoding='utf-8') as f:
        f.write("\n".join(vocabulary))
MAX_WORD_VOCABULARY_SIZE = 100000
MIN_WORD_COUNT_IN_VOCAB = 2
MAX_SEQUENCE_LEN = 50
def build_vocabulary(word_counts):
    return [wc[0] for wc in reversed(sorted(word_counts.items(), key=operator.itemgetter(1))) if wc[1] >= MIN_WORD_COUNT_IN_VOCAB and wc[0] != UNK][:MAX_WORD_VOCABULARY_SIZE] # Unk will be appended to end

In [119]:
root_path = "csv"
vocabFile = "csv/vocab.txt"
create_dev_test_train_split_and_vocabulary(root_path,"csv/test_out.csv",vocabFile)

0
comment:
['@impl', 'Note', 'taken', 'from', '{@link', 'com', 'sun', 'javafx', 'scene', 'control', 'behavior', 'Text', 'Area', 'Behavior', 'context', 'Menu', 'Requested(javafx', 'scene', 'input', 'Context', 'Menu', 'Event)}']
code:
['public', 'static', 'void', 'showContextMenu', '(', 'TextArea', 'textArea', ',', 'ContextMenu', 'contextMenu', ',', 'ContextMenuEvent', 'e', ')', '{']
1
comment:
['icon', 'set', 'Tool', 'Tip', 'Text(printed', 'View', 'Model', 'get', 'Localization())']
code:
['TABLE_ICONS', '.', 'put', '(', 'SpecialField', '.', 'PRINTED', ',', 'icon', ')', ';']
2
comment:
['Synchronize', 'changes', 'of', 'the', 'underlying', 'date', 'value', 'with', 'the', 'temporal', 'Accessor', 'Value']
code:
['BindingsHelper', '.', 'bindBidirectional', '(', 'valueProperty', '(', ')', ',', 'temporalAccessorValue', ',']
3
comment:
['Ask', 'if', 'the', 'user', 'really', 'wants', 'to', 'close', 'the', 'given', 'database']
code:
['private', 'boolean', 'confirmClose', '(', 'BasePanel', 'panel'

161
comment:
['for', 'each', 'column', 'get', 'either', 'actual', 'width', 'or', 'if', 'it', 'does', 'not', 'exist', 'default', 'value']
code:
['Map', '<', 'String', ',', 'Double', '>', 'columnWidths', '=', 'new', 'HashMap', '<', '>', '(', ')', ';']
162
comment:
['Opens', 'the', 'import', 'inspection', 'dialog', 'to', 'let', 'the', 'user', 'decide', 'which', 'of', 'the', 'given', 'entries', 'to', 'import']
code:
['private', 'void', 'addImportedEntries', '(', 'final', 'BasePanel', 'panel', ',', 'final', 'List', '<', 'BibEntry', '>', 'entries', ')', '{']
163
comment:
['Update', 'journal', 'abbreviation', 'loader']
code:
['journalAbbreviationLoader', '.', 'update', '(', 'abbreviationsPreferences', ')', ';']
164
comment:
['Generate', 'dialog', 'title']
code:
['String', 'dialogTitle', ';']
165
comment:
['Wrapper', 'around', 'one', 'of', 'our', 'actions', 'from', '{@link', 'Action}', 'to', 'convert', 'them', 'to', 'controlsfx', '{@link', 'org', 'controlsfx', 'control', 'action', 'Action}']
c

comment:
['This', 'class', 'represents', 'the', 'change', 'of', 'type', 'for', 'an', 'entry']
code:
['public', 'class', 'UndoableChangeType', 'extends', 'AbstractUndoableJabRefEdit', '{']
271
comment:
['General']
code:
[]
272
comment:
['Sets', 'the', 'title', 'of', 'the', 'main', 'window']
code:
['public', 'void', 'setWindowTitle', '(', ')', '{']
273
comment:
['check', 'mode', 'of', 'currently', 'used', 'DB']
code:
['if', '(', 'panel', '!=', 'null', ')', '{']
274
comment:
['No', 'parent', '=', 'root', '>', 'just', 'add']
code:
['this', '.', 'moveTo', '(', 'target', ')', ';']
275
comment:
['write', 'the', 'entries', 'using', 'sw', 'which', 'is', 'used', 'later', 'to', 'form', 'the', 'email', 'content']
code:
['BibEntryWriter', 'bibtexEntryWriter', '=', 'new', 'BibEntryWriter', '(']
276
comment:
['Opens', 'the', 'given', 'files', 'If', 'one', 'of', 'it', 'is', 'null', 'or', '404', 'nothing', 'happens']
code:
['public', 'void', 'openFiles', '(', 'List', '<', 'Path', '>', 'filesToOpen', ',

379
comment:
['Nothing', 'set', 'so', 'we', 'use', 'the', 'default', 'values']
code:
['while', '(', 'preferences', '.', 'get', '(', 'JabRefPreferences', '.', 'CUSTOM_TAB_NAME', '+', '"_def"', '+', 'i', ')', '!=', 'null', ')', '{']
380
comment:
['Run', 'the', 'actual', 'open', 'in', 'a', 'thread', 'to', 'prevent', 'the', 'program', 'locking', 'until', 'the', 'file', 'is', 'loaded']
code:
['BackgroundTask', '.', 'wrap', '(', '(', ')', '->', 'openIt', '(', 'file', ',', 'dialog', '.', 'importEntries', '(', ')', ',', 'dialog', '.', 'importStrings', '(', ')', ',', 'dialog', '.', 'importGroups', '(', ')', ',', 'dialog', '.', 'importSelectorWords', '(', ')', ')', ')']
381
comment:
['It', 'could', 'be', 'that', 'somehow', 'the', 'path', 'is', 'null', 'for', 'example', 'if', 'it', 'got', 'deleted', 'in', 'the', 'meantime']
code:
['if', '(', 'directory', '==', 'null', ')', '{']
382
comment:
['extract', 'url', 'parameter']
code:
['String', '[', ']', 'pairs', '=', 'query', '.', 'split', '(', '"&"',

503
comment:
['//', 'TODO', 'Show', 'detailed', 'description', 'of', 'the', 'changes', 'String', 'Builder', 'sb', '=', 'new', 'String', 'Builder(', '"<html>"', '+', 'Localization', 'lang("', 'Changes', 'have', 'been', 'made', 'to', 'the', 'following', 'metadata', 'elements")', '+', '"', '<p>', '&nbsp', '&nbsp', '")', 'sb', 'append(changes', 'stream()', 'map(unit', '>', 'unit', 'key)', 'collect(', 'Collectors', 'joining("', '&nbsp', '&nbsp', '")))', 'sb', 'append("</html>")', 'info', 'Pane', 'set', 'Text(sb', 'to', 'String())']
code:
['return', 'new', 'Label', '(', 'Localization', '.', 'lang', '(', '"Metadata change"', ')', ')', ';']
504
comment:
['Global', 'String', 'constants', 'for', 'GUI', 'actions']
code:
['@', 'Deprecated']
505
comment:
['Put', 'everything', 'together']
code:
['VBox', 'container', '=', 'new', 'VBox', '(', ')', ';']
506
comment:
['Check', 'if', 'we', 'should', 'italicize', 'the', '"et', 'al', '"', 'string', 'in', 'citations']
code:
['boolean', 'italicize', '=', 'st

...

[{'comment': [91, 81, 630, 34, 1006, 70, 109, 88, 98, 686, 539, 1227, 51, 34, 155, 34, 88, 0, 34, 299, 70, 166, 88, 715, 98, 1415, 51, 82, 1414, 51, 573, 34, 607, 242, 72, 97, 64, 462, 232, 109, 219, 0, 34, 1006, 64, 341, 1022], 'code': [67, 81, 2948, 133, 745, 35], 'label': 0}]
[{'comment': [149, 1413, 89, 976, 195, 64, 102, 869, 51, 564, 195, 239, 72, 97, 141, 204, 401, 1619, 117, 2947, 297, 596, 2946, 243, 754], 'code': [153, 2945, 49, 16, 1253, 15, 1252, 16, 971, 9, 1192, 17, 2944, 39, 17, 33], 'label': 0}]
[{'comment': [98, 186, 936, 82, 102, 681, 75, 1412, 45, 106, 1250, 140, 1104, 86, 613, 1752, 147, 535, 1005, 2943], 'code': [72, 16, 154, 15, 156, 280, 110, 17, 35], 'label': 0}]
[{'comment': [376, 2942, 185, 195], 'code': [67, 130, 2941, 16, 17, 35], 'label': 1}]
[{'comment': [1453, 34, 196, 124, 72, 97, 64, 102, 772], 'code': [67, 90, 500, 16, 17, 35], 'label': 1}]
[{'comment': [2940, 387, 51, 203, 86, 249, 76, 34, 118, 88], 'code': [67, 283, 153, 1540, 16, 130, 88, 17, 33], '

[{'comment': [167, 304, 218, 2713, 76, 361, 110, 2712], 'code': [281, 15, 2711, 16, 17, 15, 768, 16], 'label': 0}]
[{'comment': [1249, 34, 129, 103, 0, 1824, 399, 116, 34, 129, 393, 326, 647, 2710, 341, 97, 64, 1021, 51, 34, 315, 91, 1612, 608, 82, 1039], 'code': [67, 81, 2709, 35], 'label': 0}]
[{'comment': [91, 81, 1066, 4, 125, 70, 411, 138, 89, 162, 82, 417, 51, 105, 143, 142, 98, 81, 338, 2708, 272, 51, 1714, 109, 159, 4, 731, 476, 0, 4, 279, 51, 164, 275, 918, 138, 70, 86, 125], 'code': [67, 81, 1488, 193, 370, 35], 'label': 0}]
[{'comment': [1569, 360, 141, 204, 1358, 525, 72, 173, 1357, 180, 468], 'code': [72, 16, 108, 827, 15, 623, 16, 17, 15, 929, 16, 17, 15, 276, 16, 17, 17, 35], 'label': 0}]
[{'comment': [429, 245, 97, 1356, 102, 318], 'code': [536, 16, 479, 2707, 17, 35], 'label': 1}]
[{'comment': [95, 212, 940, 519, 89, 812, 199, 138, 164, 115, 102, 417, 75, 180, 70, 34, 95, 185, 2706, 70, 34, 261, 95, 185, 1373, 91, 95, 212, 940, 2705, 101, 591, 70, 560, 95, 212, 940, 51

[{'comment': [140, 542, 383, 1921, 196, 383, 123, 1920, 383, 1180, 143, 142, 383, 932, 100, 1919], 'code': [69], 'label': 0}]
[{'comment': [98, 88, 321, 356], 'code': [104, 174, 47, 130, 45, 88, 49, 174, 15, 157, 16, 17, 33], 'label': 0}]
[{'comment': [617, 4, 2458, 151, 1732, 124, 34, 2457, 151, 64, 335, 972], 'code': [80, 132, 136, 2456, 49, 35, 901, 15, 2455, 16, 17, 9, 1325, 132, 56, 136, 9, 1325, 132, 39, 136, 69, 33], 'label': 0}]
[{'comment': [600, 2454, 419, 333, 0, 203, 414, 403, 2453, 473, 70, 414, 51, 380, 480, 336, 2452], 'code': [414, 15, 2451, 16, 1837, 16, 17, 17, 33], 'label': 0}]
[{'comment': [329, 367, 838, 34, 549, 64, 102, 123, 543, 145, 1024, 837, 86], 'code': [1592, 16, 218, 9, 176, 17, 33], 'label': 0}]
[{'comment': [344, 144, 51, 493, 89, 34, 1032, 1031, 51, 123, 4, 415, 217, 89, 64, 102, 375, 75, 34, 163, 353, 409, 439], 'code': [81, 1555, 133, 344, 35], 'label': 0}]
[{'comment': [2450, 4, 172, 2449, 231, 89, 2448, 1557, 232, 34, 118, 1324], 'code': [47, 73, 45

[{'comment': [175, 205, 150, 1107, 76, 196], 'code': [2220, 16, 17, 33], 'label': 1}]
[{'comment': [2219, 34, 821, 449, 117, 516, 117, 97, 158, 149, 208, 51, 1951, 4, 310, 51, 34, 2218, 381, 34, 339, 587, 1340, 1339], 'code': [2217, 49, 882, 15, 881, 16, 1518, 15, 2216, 16, 2215, 15, 2214, 16, 17, 9, 56, 17, 17], 'label': 0}]
[{'comment': [160, 179, 2213, 0, 179, 825, 145, 120, 100, 1336, 757, 2212, 158, 51, 34, 84], 'code': [2211, 16, 17, 33], 'label': 0}]
[{'comment': [72, 34, 155, 64, 102, 157, 0, 148, 84, 64, 743, 34, 155, 1298, 105, 109, 209, 0, 141, 51, 82, 1297, 1047, 45, 698, 117, 273], 'code': [86, 15, 2210, 49, 236, 33], 'label': 0}]
[{'comment': [2209, 89, 34, 384, 986, 115, 75, 1730, 116, 103], 'code': [1294, 2208, 49, 79, 1294, 16, 797, 17, 33], 'label': 0}]
[{'comment': [253, 157, 384, 986, 72, 340, 605, 64, 2207], 'code': [2206, 49, 79, 1000, 16, 17, 33], 'label': 0}]
[{'comment': [1483, 109, 2205, 2204, 34, 527, 51, 493, 34, 155, 141, 273], 'code': [420, 15, 1690, 16, 1

[{'comment': [429, 34, 322, 1048, 86, 124, 64, 681, 75, 143, 142, 684, 2120, 1158, 245, 2119, 86, 124, 72, 713], 'code': [67, 122, 323, 156, 33], 'label': 0}]
[{'comment': [440, 339, 186], 'code': [104, 122, 1053, 1608, 33], 'label': 0}]
[{'comment': [440, 339, 186], 'code': [67, 122, 2118, 1053, 2117, 16, 17, 35], 'label': 1}]
[{'comment': [841, 601], 'code': [67, 122, 90, 2116, 16, 17, 324, 2115, 35], 'label': 1}]
[{'comment': [682, 1588, 76, 1308, 369], 'code': [2114, 15, 1531, 15, 2113, 16, 17, 33], 'label': 0}]
[{'comment': [2112, 2111, 76, 2110, 138, 0, 2109, 2108], 'code': [67, 81, 1275, 35], 'label': 0}]
[{'comment': [1543, 2107, 89, 115, 199, 144, 75, 2106, 544, 91, 64, 51, 372, 34, 22, 2105, 845, 51, 22, 56, 2104, 2103, 147, 2102, 260, 168, 402, 51, 34, 87, 923, 101, 2101, 64, 1320], 'code': [67, 122, 90, 1946, 16, 17, 35], 'label': 0}]
[{'comment': [139, 367, 1379, 679, 51, 2100, 706], 'code': [67, 122, 131, 188, 2099, 49, 1274, 92, 2098, 33], 'label': 0}]
[{'comment': [2097

In [16]:
dataframe = pd.read_csv("csv/oneLineCode.csv", na_filter = False)
for i,data in dataframe.iterrows():
    s =  data["comment"]
    c = data["code"]
    print(i)
    print("comment:")
    print(s)
    print("code:")
    print(c)

0
comment:
@implNote taken from {@link com.sun.javafx.scene.control.behavior.TextAreaBehavior#contextMenuRequested(javafx.scene.input.ContextMenuEvent)}
code:
public static void showContextMenu(TextArea textArea, ContextMenu contextMenu, ContextMenuEvent e) {
1
comment:
icon.setToolTipText(printedViewModel.getLocalization());
code:
TABLE_ICONS.put(SpecialField.PRINTED, icon);
2
comment:
Synchronize changes of the underlying date value with the temporalAccessorValue
code:
BindingsHelper.bindBidirectional(valueProperty(), temporalAccessorValue,
3
comment:
Ask if the user really wants to close the given database
code:
private boolean confirmClose(BasePanel panel) {
4
comment:
css: information *
code:
INTEGRITY_INFO(MaterialDesignIcon.INFORMATION),
5
comment:
Check if there is a default type with the same name. If so, this is a modification of that type, so remove the default one:
code:
ExternalFileType toRemove = null;
6
comment:
each entry type
code:
for (Map.Entry<EntryType, TextField> 

250
comment:
paranoia: ensure that there are never two instances of AllEntriesGroup
code:
if (newGroups.getGroup() instanceof AllEntriesGroup) {
251
comment:
Create button that triggers search
code:
Button search = new Button(Localization.lang("Search"));
252
comment:
Activate context menu if user presses the "context menu" key
code:
tableView.addEventHandler(KeyEvent.KEY_RELEASED, event -> {
253
comment:

code:
public static <V> V runInJavaFXThread(Callable<V> callable) {
254
comment:
Add the edit field for Bibtex-key.
code:
fields.add(InternalField.KEY_FIELD);
255
comment:
Remove all reference marks that don't look like JabRef citations:
code:
List<String> tmp = new ArrayList<>();
256
comment:
Although entries are redefined without use, this also updates linkSourceBase
code:
Map<BibEntry, BibDatabase> entries = findCitedEntries(databases, cited, linkSourceBase);
257
comment:
for the remaining entries, intersection has to be used this approach ensures that one empty keyword list leads

378
comment:
Process arguments
code:
ArgumentProcessor argumentProcessor = new ArgumentProcessor(arguments, ArgumentProcessor.Mode.INITIAL_START);
379
comment:
Nothing set, so we use the default values:
code:
while (preferences.get(JabRefPreferences.CUSTOM_TAB_NAME + "_def" + i) != null) {
380
comment:
Run the actual open in a thread to prevent the program locking until the file is loaded.
code:
BackgroundTask.wrap(() -> openIt(file, dialog.importEntries(), dialog.importStrings(), dialog.importGroups(), dialog.importSelectorWords()))
381
comment:
It could be that somehow the path is null, for example if it got deleted in the meantime
code:
if (directory == null) {
382
comment:
extract url parameter
code:
String[] pairs = query.split("&");
383
comment:
Collect all callables to execute in one collection.
code:
for (BibEntry entry : entries) {
384
comment:
TODO: Add undo Store undo information. AbstractUndoableEdit undoAddPreviousEntries = null; UndoableModifyGroup undo = new UndoableModi

TABLE_ICONS.put(StandardField.PDF, icon);
575
comment:
First get a list of the default file types as a starting point:
code:
List<ExternalFileType> types = new ArrayList<>(getDefaultExternalFileTypes());
576
comment:
The name to change to is already in the database, so we can't comply.
code:
LOGGER.info("Cannot rename string '" + mem + "' to '" + disk + "' because the name "
577
comment:
Look up the external file type registered with this name, if any.
code:
public Optional<ExternalFileType> getExternalFileTypeByName(String name) {
578
comment:
TODO: Add undo/redo ce.addEdit(new UndoableInsertEntry(panel.getDatabase(), entry));
code:

579
comment:
The action concerned with generate a new (sub-)database from latex AUX file.
code:
public class NewSubLibraryAction extends SimpleCommand {
580
comment:
Starts the search of unlinked files according chosen directory and the file type selection. The search will process in a separate thread and a progress indicator will be displayed.
code:
priv

public <T> BackgroundTask<T> thenRun(Function<V, T> nextOperation) {
775
comment:
This action checks whether any new custom entry types were loaded from this BIB file. If so, an offer to remember these entry types is given.
code:
public class CheckForNewEntryTypesAction implements GUIPostOpenAction {
776
comment:

code:
private void openTheFile(Path file, boolean raisePanel) {
777
comment:
Other fields
code:
entryEditorTabs.add(new OtherFieldsTab(databaseContext, panel.getSuggestionProviders(), undoManager,
778
comment:
Register listener The wrapper created by the FXCollections will set a weak listener on the wrapped list. This weak listener gets garbage collected. Hence, we need to maintain a reference to this list.
code:
entriesList = databaseContext.getDatabase().getEntries();
779
comment:
Try to download fulltext PDF for selected entry(ies) by following URL or DOI link.
code:
public class FindFullTextAction extends SimpleCommand {
780
comment:
Editor for external file types.
code:


code:
public static JabRefPreferences prefs;
975
comment:
Key binding preferences
code:
private static KeyBindingRepository keyBindingRepository;
976
comment:
Key binding preferences
code:
public static synchronized KeyBindingRepository getKeyPrefs() {
977
comment:
Background tasks
code:
public static void startBackgroundTasks() throws JabRefException {
978
comment:
FIXME: Workaround for bug https://github.com/Microsoft/ApplicationInsights-Java/issues/662
code:
SDKShutdownActivity.INSTANCE.stopAll();
979
comment:
Static variables for graphics files and keyboard shortcuts.
code:
public class GUIGlobals {
980
comment:
Perform initializations that are only used in graphical mode. This is to prevent the "Xlib: connection to ":0.0" refused by server" error when access to the X server on Un*x is unavailable.
code:
public static void init() {
981
comment:
add some additional space to improve appearance
code:
public static final int WIDTH_ICON_COL = 16 + 12;
982
comment:
Width of Ranking Icon 

In [54]:
delimiters = "#", ".",",","<b>","</b>","-",":","<br>","_","?"," ",";"
example = "stacko#verFlow (c) is a=Wesome... isn't it? D/?DD"
def splitText(delimiters, string, maxsplit=0):
    string.replace()
    import re
    regexPattern = '|'.join(map(re.escape, delimiters))
#     print(regexPattern)
    result =  re.split(regexPattern, string, maxsplit)
    f = []
    for r in result:
        p =  re.sub(r'((?<=[a-z])[A-Z]|(?<!\A)[A-Z](?=[a-z]))', r' \1', r)
#         print(p)
        for q in p.split():
            if q != "":
                f.append(q)
    return f
splitText(delimiters,example)

['stacko',
 'ver',
 'Flow',
 '(c)',
 'is',
 'a=',
 'Wesome',
 "isn't",
 'it',
 'D/',
 'DD']

In [60]:
tokens = javalang.tokenizer.tokenize('System.out.println("Hello " + "world");')
parser = javalang.parser.Parser(tokens)
parser.parse_expression()

MethodInvocation(arguments=[BinaryOperation(operandl=Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="Hello "), operandr=Literal(postfix_operators=[], prefix_operators=[], qualifier=None, selectors=[], value="world"), operator=+)], member=println, postfix_operators=[], prefix_operators=[], qualifier=System.out, selectors=[], type_arguments=None)

In [73]:
f = "This class monitors a set of files for changes. Upon detecting a change it notifies the registered {@link FileUpdateListener}s. Implementation based on https://stackoverflow.com/questions/16251273/can-i-watch-for-single-file-change-with-watchservice-not-the-whole-directory"
splitComment(f)

https://stackoverflow.com/questions/16251273/can-i-watch-for-single-file-change-with-watchservice-not-the-whole-directory


['This',
 'class',
 'monitors',
 'a',
 'set',
 'of',
 'files',
 'for',
 'changes',
 'Upon',
 'detecting',
 'a',
 'change',
 'it',
 'notifies',
 'the',
 'registered',
 '{@link',
 'File',
 'Update',
 'Listener}s',
 'Implementation',
 'based',
 'on',
 'https']